In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/home/torsten/ms/results/with_gnss.csv", header=None)
df

,0,1,2,3,4
0,1,0,0.000154,0.000412,0.000061
1,1,1,-0.996860,0.177555,-7.236810
2,2,0,-0.116728,-0.028926,-0.144100
3,2,1,0.545500,0.135718,-0.326219
4,2,2,-5.850210,-1.823070,-3.633040
...,...,...,...,...,...
20295,200,196,9.119240,-5.890200,-7.428120
20296,200,197,10.642000,-6.841700,-7.346440
20297,200,198,12.822900,-7.997830,-7.199360
20298,200,199,14.890500,-7.863380,-7.038390


In [3]:
%matplotlib tk
start = 0

plt.figure()
plt.axis('equal')
plt.grid()
for i in range(200):
    plt.plot(df.iloc[start:start+i+2, 2], df.iloc[start:start+i+2, 3], "-x")
    plt.text(df.iloc[start+i+1, 2], df.iloc[start+i+1, 3], str(i+1))
    
    start+=i+2

plt.show()